In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('CKD.csv')

In [3]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [4]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset['classification_yes']

In [5]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [9]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test) 

In [10]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

In [11]:
print(cm)

[[51  0]
 [ 1 81]]


In [12]:
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [13]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.9924946382275899


In [15]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [16]:
table=pd.DataFrame.from_dict(re)

In [17]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015111,0.003117,0.004160,0.002146,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
1,0.014452,0.008688,0.002535,0.003285,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.981569,0.981014,0.981217,1.000000,1.000000,0.988760,0.009179,1
2,0.003311,0.003744,0.008828,0.003196,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.963284,0.981233,0.962573,0.981217,0.981217,0.973905,0.008965,4
3,0.022647,0.005109,0.000000,0.000000,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.981569,0.981233,0.981217,0.981217,0.981217,0.981291,0.000139,3
